In [ ]:
from platform import machine

import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, Normalizer, FunctionTransformer

from sklearn.tree import DecisionTreeClassifier

In [ ]:
df = pd.read_csv('archive/weather_features.csv')
df

# 1
Si vuole predire il valore di 'weather_main' sulla base degli attributi presenti nel dataset.
Dividere il dataset in modo che 2/3 degli elementi siano contenuti in un nuovo dataset “train” e
1/3 nel dataset “test”.
Eliminare gli attributi ["dt_iso","city_name","weather_description", "weather_icon","weather_id",
"clouds_all"]
Convertire l’attributo 'weather_main' in numerico in maniera opportuna.
Allenare il train con il modello Decision Tree e valutare l’accuracy ottenuta calcolata sia sul
dataset train sia sul dataset test. Confrontare i risultati ottenuti con quelli ottenuti con una
predizione basata sul modello Logistic Regression. Effettuare alcune considerazioni sui risultati
ottenuti, tenendo in considerazione anche l’analisi della confusion matrix. (punti 4)

In [ ]:
X = df.drop(['dt_iso', 'city_name', 'weather_description', 'weather_icon', 'weather_id', 'clouds_all', 'weather_main'], axis=1)
le = LabelEncoder()
df['weather_target'] = le.fit_transform(df['weather_main'])
y = df['weather_target']
y

In [71]:
X

,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_3h
0,270.475,270.475,270.475,1001,77,1,62,0.0,0.0,0.0
1,270.475,270.475,270.475,1001,77,1,62,0.0,0.0,0.0
2,269.686,269.686,269.686,1002,78,0,23,0.0,0.0,0.0
3,269.686,269.686,269.686,1002,78,0,23,0.0,0.0,0.0
4,269.686,269.686,269.686,1002,78,0,23,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
178391,287.760,287.150,288.150,1028,54,3,30,0.0,0.0,0.0
178392,285.760,285.150,286.150,1029,62,3,30,0.0,0.0,0.0
178393,285.150,285.150,285.150,1028,58,4,50,0.0,0.0,0.0
178394,284.150,284.150,284.150,1029,57,4,60,0.0,0.0,0.0


In [72]:
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.33, random_state=42)

dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

y_pred = dt.predict(X_train)
acc_dt_train = accuracy_score(y_train, y_pred)
print(f"--- Decision Tree ---")
print(f' Accuracy dt train: {acc_dt_train}')
y_pred = dt.predict(X_test)
acc_dt_test = accuracy_score(y_test, y_pred)
print(f' Accuracy dt test: {acc_dt_test}')
print(confusion_matrix(y_test, y_pred))

--- Decision Tree ---
 Accuracy dt train: 0.98399498012968
 Accuracy dt test: 0.6366632127872807
[[18522  7875    90    63   239    70   301     0    12    11    57]
 [ 7913 13625   216    28   208    30   349     0     3    17   122]
 [   75   170    38     1    42     2    90   160     1     2     7]
 [   66    25     1    15     2     2     3     5     0     0     0]
 [  181   192    43     4   163     7   217    16     0     7     1]
 [   57    42     0     2    12    15     5    10     0     0     0]
 [  262   332   110     0   256    11   146   130     0     6     8]
 [    0     0   326    12    35     9   242  4929     0    31   138]
 [    7     2     0     0     0     0     0     0     0     0     0]
 [    8    12     9     0     4     0    10    27     0    17     0]
 [   65   114     6     0     1     0    12   151     0     0    11]]


In [73]:
lr = LogisticRegression(max_iter=100)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_train)
acc_lr_train = accuracy_score(y_train, y_pred)
print(f"--- Logistic Regression ---")
print(f' Accuracy lr train: {acc_lr_train}')
y_pred = lr.predict(X_test)
acc_lr_test = accuracy_score(y_test, y_pred)
print(f' Accuracy lr test: {acc_lr_test}')
print(confusion_matrix(y_test, y_pred))

--- Logistic Regression ---
 Accuracy lr train: 0.4798326709893328
 Accuracy lr test: 0.4796589152553889
[[18900  8340     0     0     0     0     0     0     0     0     0]
 [13174  9337     0     0     0     0     0     0     0     0     0]
 [   70   518     0     0     0     0     0     0     0     0     0]
 [   84    35     0     0     0     0     0     0     0     0     0]
 [   58   773     0     0     0     0     0     0     0     0     0]
 [   45    98     0     0     0     0     0     0     0     0     0]
 [   65  1196     0     0     0     0     0     0     0     0     0]
 [ 1155  4566     0     0     0     0     0     1     0     0     0]
 [    7     2     0     0     0     0     0     0     0     0     0]
 [    6    81     0     0     0     0     0     0     0     0     0]
 [  148   212     0     0     0     0     0     0     0     0     0]]


/Users/gabrieledavena/Uni/Big Data/BData/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Il Decision Tree supera la Logistic Regression sul Test set (0.63 vs 0.50), dimostrando che la relazione tra le features e il tipo di meteo non è lineare. Tuttavia, la LR soffre meno di overfitting.

# 2
Confrontare l’accuratezza ottenuta nel punto precedente con l’accuratezza che si ottiene con
un una 10 Fold cross validation. (punti 1)

In [74]:
cross_val = cross_val_score(dt, X, y, cv=5, scoring='accuracy')
print(f'Cross val: {cross_val.mean()}')

/Users/gabrieledavena/Uni/Big Data/BData/lib/python3.13/site-packages/sklearn/model_selection/_split.py:811: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Cross val: 0.5253760381400322


# 3
Utilizzare la funzione Normalizer per normalizzare i valori del dataset e confrontare se
l’accuratezza ottenuta con il Decision Tree Classifier migliora (punti 3).

In [75]:
norm = Normalizer()
X = norm.fit_transform(X)

X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.33, random_state=42)
dt.fit(X_train, y_train)
y_pred = dt.predict(X_train)
acc_dt_train = accuracy_score(y_train, y_pred)
print(f"--- Decision Tree ---")
print(f' Accuracy dt train: {acc_dt_train}')
y_pred = dt.predict(X_test)
acc_dt_test = accuracy_score(y_test, y_pred)
print(f' Accuracy dt test: {acc_dt_test}')
print(confusion_matrix(y_test, y_pred))

--- Decision Tree ---
 Accuracy dt train: 0.98399498012968
 Accuracy dt test: 0.6210528103820218
[[17964  8416    62    68   240    88   309     0     9    16    68]
 [ 8213 13307   184    31   245    51   331     0    12     7   130]
 [   72   181    27     0    46     3    92   158     0     1     8]
 [   60    38     1    12     2     0     0     6     0     0     0]
 [  199   179    42     4   162     8   209    16     0    11     1]
 [   56    51     1     1    10     9     5    10     0     0     0]
 [  287   341   100     1   246    13   130   130     0     6     7]
 [    0     0   346    13    39     9   232  4928     0    27   128]
 [    6     3     0     0     0     0     0     0     0     0     0]
 [   11     9     6     0     4     0    10    33     0    14     0]
 [   59   118     8     1     2     2    10   150     0     1     9]]


# 4
Creare una pipeline con trasformatori PCA (si scelgano 5 attributi) e poi Normalizer. Si usi
come modello il Decision Tree Classifier (punti 2) [2 punti ulteriori se gli attributi della PCA
sono aggiunti agli attributi del dataset]

In [76]:
identity_transformer = FunctionTransformer(lambda x: x, validate=False)

combined_features = FeatureUnion([
    ("original_features", identity_transformer),  # Ramo 1: Dati originali
    ("pca_features", PCA(n_components=5))       # Ramo 2: 5 attributi PCA
])

pipeline = Pipeline([
    ('combined_features', combined_features),
    ('normalizer', Normalizer()),
    ('dt', DecisionTreeClassifier()),
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_train)
acc_PCA_dt_train = accuracy_score(y_train, y_pred)
print(f"--- PCA COMBINED ---")
print(f' Accuracy dt train: {acc_dt_train}')
y_pred = pipeline.predict(X_test)
acc_PCA_dt_test = accuracy_score(y_test, y_pred)
print(f' Accuracy dt test: {acc_PCA_dt_test}')

--- PCA COMBINED ---
 Accuracy dt train: 0.98399498012968
 Accuracy dt test: 0.6265223964260842


PCA(n_components=5): Estrae le 5 componenti che spiegano la maggior varianza. Aggiungendole al dataset originale, stiamo fornendo al Decision Tree delle "meta-informazioni" riassuntive che potrebbero aiutarlo a fare tagli migliori.

# 5
Utilizzare la funzione di gridSearchCV sulla pipeline per modificare il numero di attributi
selezionati dalla PCA e alcuni parametri a piacere del classificatore. Verificare se l’accuratezza
che si ottiene con la nuova configurazione supera quella standard ottenuta al punto 1 (punti 4)

In [77]:
param_grid ={
    'combined_features__pca_features__n_components' : [3, 4, 5, 6, 7],
    'dt__criterion' : ['gini', 'entropy'],
    'dt__max_depth' : [3, 5, 7],
}
gs = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
gs.fit(X_train, y_train)
y_pred = gs.predict(X_test)
acc_gs = accuracy_score(y_test, y_pred)
print(f" Accuracy grid search test: {acc_gs}")
print(f'Best params: {gs.best_params_}')
print(f'Best score: {gs.best_score_}')

print(f"\n--- Confronto ---")
print(f"Accuracy Standard (Punto 1): {acc_dt_test:.4f}") # Assicurati che questa var esista ancora
print(f"Accuracy Ottimizzata: {acc_gs:.4f}")

if acc_gs > acc_dt_test:
    print("RISULTATO: La nuova configurazione ha migliorato le performance!")
else:
    print("RISULTATO: Non c'è stato un miglioramento (o l'overfitting è stato ridotto a scapito dell'accuracy pura).")

/Users/gabrieledavena/Uni/Big Data/BData/lib/python3.13/site-packages/sklearn/model_selection/_split.py:811: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


 Accuracy grid search test: 0.643848414329636
Best params: {'combined_features__pca_features__n_components': 5, 'dt__criterion': 'gini', 'dt__max_depth': 7}
Best score: 0.6433884124660112

--- Confronto ---
Accuracy Standard (Punto 1): 0.6211
Accuracy Ottimizzata: 0.6438
RISULTATO: La nuova configurazione ha migliorato le performance!


# 6 NON POSSO FARLO; NON HO I DATASET

# 7
Si sperimenti una pipeline come quella del punto 4 dove al posto del classificatore si utilizzi
un regressore lineare. Il risultato dovrà essere approssimato all’intero per il calcolo
dell’accuratezza (punti 2).

In [81]:
identity_transformer = FunctionTransformer(lambda x: x, validate=False)

combined_features = FeatureUnion([
    ("original_features", identity_transformer),  # Ramo 1: Dati originali
    ("pca_features", PCA(n_components=5))       # Ramo 2: 5 attributi PCA
])

pipeline_reg = Pipeline([
    ('combined_features', combined_features),
    ('normalizer', Normalizer()),
    ('dt', LinearRegression()),
])

pipeline_reg.fit(X_train, y_train)
y_pred = np.round(pipeline_reg.predict(X_train)).astype(int)
acc_PCA_reg_train = accuracy_score(y_train, y_pred)
print(f"--- PCA COMBINED REG ---")
print(f' Accuracy reg train: {acc_PCA_reg_train}')
y_pred = np.round(pipeline.predict(X_test)).astype(int)
acc_PCA_reg_test = accuracy_score(y_test, y_pred)
print(f' Accuracy dt test: {acc_PCA_reg_test}')
print(confusion_matrix(y_test, y_pred))

--- PCA COMBINED REG ---
 Accuracy reg train: 0.31612633340305374
 Accuracy dt test: 0.6265223964260842
[[18169  8145    95    73   269    72   333     0    10    11    63]
 [ 8175 13427   193    24   211    43   330     0     9     7    92]
 [   88   174    35     1    35     1    87   159     0     1     7]
 [   66    28     3    13     2     0     1     6     0     0     0]
 [  192   192    41     2   157     2   224    15     0     6     0]
 [   58    44     1     1    12    11     5    11     0     0     0]
 [  284   342   106     0   246     7   130   136     0     4     6]
 [    0     1   339    14    38     8   249  4923     0    26   124]
 [    5     3     0     0     0     1     0     0     0     0     0]
 [   11    14     6     0     4     0    11    30     0    10     1]
 [   61   121     7     2     0     0    12   148     0     0     9]]


La linear regression dovrebbe avere risultati peggiori perché noi stiamo facendo classificazione, ma lui la vede come una regressione, quindi pensa che cloud(2) è il doppio di soleggiato (1), ma in realtà sono cose che non c'entrano un cazzo